# Setup PySpark

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
import pyspark.sql.functions as F
import os
import pandas as pd
import random
import hashlib
import itertools

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#os.environ['PYSPARK_SUBMIT_ARGS'] = \
#  '--org.postgresql.Driver /Users/tbiytc/Desktop/LifeZ_Data/postgresql-42.4.1.jar pyspark-shell'
#sc = SparkContext()
conf = SparkConf()  # create the configuration
conf.setMaster("local")
conf.set("spark.jars", "/Users/tbiytc/Desktop/LifeZ_Data/postgresql-42.4.1.jar")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark = SparkSession.builder\
                    .config(conf = conf)\
                    .appName('test').getOrCreate()
sc = SparkContext.getOrCreate()

22/09/04 17:24:47 WARN Utils: Your hostname, Jasons-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.2.47 instead (on interface en0)
22/09/04 17:24:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/04 17:24:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark

---

## Names & Entities Definition

In [3]:
# Set up the database names and accesses
db_name = "medical"
db_usrname = "tbiytc"
db_pssword = "jason1995"

---

## Data Extraction

In [23]:
# List of files
source_dir = "./Synthea/csv/"
sources = [source_dir+x for x in os.listdir(source_dir) if x.endswith(".csv")]
sources

['./Synthea/csv/medications.csv',
 './Synthea/csv/providers.csv',
 './Synthea/csv/payer_transitions.csv',
 './Synthea/csv/imaging_studies.csv',
 './Synthea/csv/payers.csv',
 './Synthea/csv/allergies.csv',
 './Synthea/csv/procedures.csv',
 './Synthea/csv/organizations.csv',
 './Synthea/csv/conditions.csv',
 './Synthea/csv/careplans.csv',
 './Synthea/csv/encounters.csv',
 './Synthea/csv/devices.csv',
 './Synthea/csv/immunizations.csv',
 './Synthea/csv/patients.csv',
 './Synthea/csv/observations.csv']

## Data Transformation

In [126]:
# AGG - df_agg
spark_dagg_df = spark.createDataFrame(df_agg)

/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:425: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


## Data Loading

In [20]:
import pandas as pd
import os

def write_to_db(data_frame, table_name):
    data_frame.write.format('jdbc').options(
              url=f'jdbc:postgresql://localhost:5432/{db_name}',
              driver='org.postgresql.Driver',
              dbtable=table_name,
              user=db_usrname,
              password=db_pssword).mode('append').save()
    out_message = f"Data frame {data_frame} has been appended to table {table_name} in the PostgreSQL database."
    return out_message

In [24]:
# Loop
for s in sources:
    sp_tmp = spark.read.format("csv").options(header="true").load(s)
    # Load into db
    write_to_db(sp_tmp, s.split('/')[-1].split('.')[0])

---

In [52]:
clinical_data = [x for x in os.listdir("./Glioma/CGGA/")]

In [53]:
c_data = pd.DataFrame()
for c in clinical_data:
    c_tmp = pd.read_csv("./Glioma/CGGA/"+c, sep='\t')
    c_data = pd.concat([c_data, c_tmp], axis=0).reset_index(drop=True)

In [55]:
c_data.query("Gender=='Female' & Age < 27 & Age > 20 & `Censor (alive=0; dead=1)` != 0", engine="python").sort_values('Age')

,CGGA_ID,Subtype,Histology,Grade,Gender,Age,OS,Censor (alive=0; dead=1),Radio_status (treated=1;un-treated=0),Chemo_status (TMZ treated=1;un-treated=0),IDH_mut_status,1p19q_codeletion_status,MGMTp_methylation_status,TCGA_subtypes,PRS_type,IDH_mutation_status,1p19q_Codeletion_status,Sample_ID,WHO Grade,IDH1 R132 mutation,MGMT promoter methylation,TERT promoter C228T,TERT promoter C250T,1p/19q co-del,Sample point,Sample type,Contrast enhancement,Sequenced cells,QC passed cells,QC passed rate
229,CGGA_P181,Primary LGG,AA,WHO III,Female,21.0,238.0,1.0,NaN,NaN,Wildtype,Non-codel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1384,CGGA_P181,NaN,AA,WHO III,Female,21.0,238.0,1.0,NaN,NaN,NaN,Non-codel,NaN,NaN,Primary,Wildtype,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,CGGA_1351,Recurrent LGG,rAA,WHO III,Female,22.0,NaN,NaN,1.0,1.0,Wildtype,NaN,un-methylated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1427,CGGA_P356,NaN,rA,WHO II,Female,22.0,569.0,1.0,1.0,1.0,NaN,Non-codel,methylated,NaN,Recurrent,Mutant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407,CGGA_P29,NaN,rAOA,WHO III,Female,22.0,103.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,Recurrent,Mutant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476,CGGA_504,NaN,GBM,WHO IV,Female,22.0,563.0,1.0,1.0,0.0,NaN,NaN,methylated,Classical,Primary,Wildtype,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
946,CGGA_1425,NaN,GBM,WHO IV,Female,22.0,640.0,1.0,1.0,1.0,NaN,Codel,methylated,NaN,Primary,Wildtype,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,CGGA_504,NaN,GBM,WHO IV,Female,22.0,563.0,1.0,1.0,0.0,NaN,NaN,methylation,NaN,Primary,Wildtype,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1457,CGGA_P568,NaN,rAO,WHO III,Female,23.0,218.0,1.0,1.0,1.0,NaN,Codel,methylated,NaN,Recurrent,Mutant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1312,CGGA_881,NaN,rAA,WHO III,Female,23.0,252.0,1.0,0.0,1.0,NaN,Non-codel,NaN,NaN,Recurrent,Wildtype,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
